In [ ]:
#will build a simple neural network
#train on real world dataset
#will mimic pytotch workflow
#will have manual elements
#end result in not important

In [53]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [51]:
df = pd.read_csv('/content/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [54]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)

In [55]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [57]:
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2)

In [59]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [60]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

Numpy Array to Pytorch Tensor

In [61]:
X_train_tensor = torch.from_numpy(X_train_scaled)
X_test_tensor = torch.from_numpy(X_test_scaled)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

Defining The Model

In [71]:
class MySimpleNN():
  def __init__(self,X):
    self.weights = torch.rand(X.shape[1],1,dtype=torch.float64,requires_grad=True)
    self.bias = torch.zeros(1,dtype=torch.float64,requires_grad=True)

  def forward_pass(self,X):
    z = torch.matmul(X,self.weights) + self.bias
    y_pred = torch.sigmoid(z)
    return y_pred

  def loss_function(self,y_pred,y):
    #Clamp Prediction to avoid log0
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred,epsilon,1-epsilon)

    #Calculate Loss
    loss = -(y_train_tensor*torch.log(y_pred) + (1-y_train_tensor)*torch.log(1-y_pred)).mean()
    return loss




Important Parameters

In [82]:
learning_rate = 0.01
epochs = 25

Training Pipeline

In [83]:
#create model
model = MySimpleNN(X_train_tensor)

#define loop
for epoch in range(epochs):

  #forward pass
  y_pred = model.forward_pass(X_train_tensor)

  #loss calculate
  loss = model.loss_function(y_pred,y_train_tensor)

  #backward pass
  loss.backward()

  #parameters update
  with torch.no_grad():
    model.weights -= learning_rate*model.weights.grad
    model.bias -= learning_rate*model.bias.grad

  #clearing gradients
  model.weights.grad.zero_()
  model.bias.grad.zero_()

  #print loss in each epoch
  print(f"Epoch:{epoch+1}, Loss:{loss.item()}")


Epoch:1, Loss:4.148053291312868
Epoch:2, Loss:4.13689190229536
Epoch:3, Loss:4.125735351600294
Epoch:4, Loss:4.114583698201658
Epoch:5, Loss:4.10343700182085
Epoch:6, Loss:4.092254505618031
Epoch:7, Loss:4.08102000966187
Epoch:8, Loss:4.069632145690082
Epoch:9, Loss:4.05824966051731
Epoch:10, Loss:4.046872620469883
Epoch:11, Loss:4.035501092703781
Epoch:12, Loss:4.024122904805576
Epoch:13, Loss:4.012577233845407
Epoch:14, Loss:4.001037375833965
Epoch:15, Loss:3.9895034032300463
Epoch:16, Loss:3.977943488904666
Epoch:17, Loss:3.9662502483440343
Epoch:18, Loss:3.954416723752099
Epoch:19, Loss:3.9425553353357823
Epoch:20, Loss:3.9304612020617706
Epoch:21, Loss:3.918169191259419
Epoch:22, Loss:3.9058783888229427
Epoch:23, Loss:3.8933791577331722
Epoch:24, Loss:3.880742175596913
Epoch:25, Loss:3.8680511913639943


Evaluatio

In [85]:
with torch.no_grad():
  y_pred = model.forward_pass(X_test_tensor)
  y_pred = (y_pred>0.5).float()

accuracy = (y_pred == y_test_tensor).float().mean()
print(f"Accuracy : {accuracy.item()}")



Accuracy : 0.5627885460853577
